# ETL of the Demand Script

<a id='intro'></a>
## Introduction

Before its analysis, the data from the Demand files needs some transformations. In this script I'll be doing all of those transformations, explained step by step.

<a id='wrangling'></a>
## Structure and characteristics of the dataset

The data from the Demand is structured in 730 files, one for each day. Each file contains data from 09:00 PM of the previous day to 02:55 of the next day. That creates a lot of duplicates that will have to be removed.

The dataset contains 4 columns: 

* The first one is the date and time, every 5 minutes. 

* The second one, called 'Real', contains the actual value of energy demand for each date and time. 

* The column 'Prevista' is elaborated by REE directly. It contains the expected energy demand for every date and time and it's calculated with the values of the consumption of similar periods. It includes some corrections due to any factor that may affect the consumption, like the weather or economic activity.

* The column 'Programada' contains the scheduled energy demand for every date and time. It's elaborated for those groups/actors that are involved in the energy production.

### General Properties

#### Import libraries

In [1]:
import pandas as pd
import numpy as np
import glob
from datetime import datetime

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Import and print dataset

In [3]:
# Glob module finds all the pathnames matching a specified pattern, although results are returned in arbitrary order
path = "/content/drive/MyDrive/Data REE/XLSX - DEMANDA"
df = pd.concat(map(pd.read_excel, glob.glob(path + "/*.xlsx")))
df.head()

,Seguimiento de la demanda de energía eléctrica (MW),Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,NaN,NaN,NaN
1,Hora,Real,Prevista,Programada
2,2021-07-31 21:00:00,29496.0,29688.0,30296.0
3,2021-07-31 21:05:00,29510.0,29658.0,30296.0
4,2021-07-31 21:10:00,30006.0,29642.0,30296.0


In [4]:
# There are a lot of duplicates to remove. Reseting index is always needed after dropping duplicates.
df = df.drop_duplicates()
df.reset_index(inplace = True)

# The "index" column should be dropped.
df.drop("index", axis = 1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


The format of the columns is wrong, it's the second row the one that contains the true names of the columns.

In [5]:
# Create a list with the elements that should not be the column titles.
not_column = list(df.columns)
not_column

['Seguimiento de la demanda de energía eléctrica (MW)',
 'Unnamed: 1',
 'Unnamed: 2',
 'Unnamed: 3']

In [6]:
# Create a list with the elements that should be the column titles.
ls = df.values.tolist()
column = ls[1]
column

['Hora', 'Real', 'Prevista', 'Programada']

In [7]:
# Use those lists to change the column titles properly.
df = df.rename(columns={not_column[i]: column[i] for i in range(len(not_column))}) 

In [8]:
# Remove the last row of every file. These are the rows that have a value in 'Real', but not in 'Prevista' nor 'Programada'.
indexes = []
a = list(df["Prevista"].isna())
b = list(df["Real"].isna())
for i in range(len(a)):
  if a[i] == True and b[i] == False:
    indexes.append(i)
len(indexes)

727

In [9]:
df = df.drop(df.index[indexes])

In [10]:
# Remove the first and second row, since they don't have useful information anymore.
df = df.drop([df.index[0] , df.index[1]])
df.tail()

,Hora,Real,Prevista,Programada
210963,2020-08-09 20:35:00,29721.0,29721.0,29796.0
210964,2020-08-09 20:40:00,29788.0,29820.0,29796.0
210965,2020-08-09 20:45:00,29917.0,29934.0,29796.0
210966,2020-08-09 20:50:00,30072.0,30064.0,29796.0
210967,2020-08-09 20:55:00,29963.0,30210.0,29796.0


In [11]:
# Convert the 'Hora' column into string to solve the '2A' and '2B' issue.
df['Hora'] = df['Hora'].astype(str)

In [12]:
df.reset_index(inplace = True)
df.drop("index", axis = 1, inplace = True)

In [13]:
# Create a list with the indexes of the rows that contain 2A or 2B. 
indexes_b = []
indexes_a = []
for i in range(df.shape[0]):
  if type(df.iloc[i][0]) == str:
    if "B" in df.iloc[i][0]:
      indexes_b.append(i)
    if "A" in df.iloc[i][0]:
      indexes_a.append(i)

In [14]:
# Drop the rows with '2B'
df = df.drop(df.index[indexes_b], axis = 0)

In [15]:
# Replace '2A' by '02'
df['Hora'] = df['Hora'].str.replace("2A", "02")
df.tail()

,Hora,Real,Prevista,Programada
210235,2020-08-09 20:35:00,29721.0,29721.0,29796.0
210236,2020-08-09 20:40:00,29788.0,29820.0,29796.0
210237,2020-08-09 20:45:00,29917.0,29934.0,29796.0
210238,2020-08-09 20:50:00,30072.0,30064.0,29796.0
210239,2020-08-09 20:55:00,29963.0,30210.0,29796.0


In [16]:
# With the correct format, I convert 'Hora' into datetime
df['Hora'] = df['Hora'].astype('datetime64[ns]')
df.sort_values(by = "Hora", inplace = True)
df.head()

,Hora,Real,Prevista,Programada
208488,2020-08-01 00:00:00,31338.0,31577.0,31321.0
208489,2020-08-01 00:05:00,31622.0,31465.0,31321.0
208490,2020-08-01 00:10:00,31711.0,31348.0,31321.0
208491,2020-08-01 00:15:00,31645.0,31224.0,31321.0
208492,2020-08-01 00:20:00,31471.0,31096.0,31321.0


In [17]:
# Create different columns for date and time
df['date'] = df['Hora'].dt.date 
df["hour"] = df["Hora"].dt.time

# Drop the 'Hora' column since now is useless
df.drop("Hora", axis = 1, inplace = True) 
df.head()

,Real,Prevista,Programada,date,hour
208488,31338.0,31577.0,31321.0,2020-08-01,00:00:00
208489,31622.0,31465.0,31321.0,2020-08-01,00:05:00
208490,31711.0,31348.0,31321.0,2020-08-01,00:10:00
208491,31645.0,31224.0,31321.0,2020-08-01,00:15:00
208492,31471.0,31096.0,31321.0,2020-08-01,00:20:00


Now that we took a column with time and date the columns "Hora" and "Time" are useless. We can remove them.

In [18]:
# Convert the 'date' column into datetime
df["date"] = df["date"].astype('datetime64[ns]')

# Convert the 'Real', 'Prevista' and 'Programada' column values into int.
numbers = df.columns[0:3]
df[numbers] = df[numbers].astype(int) 

# Change the order of the columns
new_order = ["date","hour","Real","Prevista","Programada"]
df = df[new_order]
df.head()

,date,hour,Real,Prevista,Programada
208488,2020-08-01,00:00:00,31338,31577,31321
208489,2020-08-01,00:05:00,31622,31465,31321
208490,2020-08-01,00:10:00,31711,31348,31321
208491,2020-08-01,00:15:00,31645,31224,31321
208492,2020-08-01,00:20:00,31471,31096,31321


Next step should be translating the titles into English.

In [19]:
# Translate into English
not_translated = ["Real","Prevista","Programada"] 
translated = ["real","expected","scheduled"]
df = df.rename(columns={not_translated[i]: translated[i] for i in range(len(translated))})

# Reset index and erase the 'index' column.
df = df.reset_index() 
df = df.drop(["index"],axis = 1)
df.head()

,date,hour,real,expected,scheduled
0,2020-08-01,00:00:00,31338,31577,31321
1,2020-08-01,00:05:00,31622,31465,31321
2,2020-08-01,00:10:00,31711,31348,31321
3,2020-08-01,00:15:00,31645,31224,31321
4,2020-08-01,00:20:00,31471,31096,31321


In [20]:
# Prove that every date and hour have 1 value, that there are no duplicates.
dfx = df.groupby(['date','hour']).count()
dfx = dfx[dfx['real'] != 1]
dfx

,,real,expected,scheduled
date,hour,,,


In [21]:
# Convert the dataset into a csv file
df.to_csv("demand_script.csv", index = False)

In [22]:
# Importing libraries to connect with Google BigQuery
from google.cloud import bigquery
from google.oauth2 import service_account

# Adding the credentials from the JSON file to connect with the platform
credentials = service_account.Credentials.from_service_account_file(
'/content/drive/MyDrive/Data REE/energy-ree-project-282e3f1ab60e.json')
project_id = 'energy-ree-project'
client = bigquery.Client(credentials= credentials, project=project_id)

In [28]:
# Set table_id to the ID of the table to create
table_id = 'energy-ree-project.energy_generic_info.energy-demand-ree'

# Add the schema of the table
schema = [
    bigquery.SchemaField("date", "DATE", mode="NULLABLE"),
    bigquery.SchemaField("hour", "TIME", mode="NULLABLE"),
    bigquery.SchemaField("real", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("expected", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("scheduled", "INTEGER", mode="NULLABLE"),

]

# Make an API request to create the table
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)  
print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))

Created table energy-ree-project.energy_generic_info.energy-demand-ree


In [29]:
# Specify the name of the file
file_path = 'demand_script.csv'

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1, autodetect=True,
)

# Open and read the file
with open(file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id, job_config=job_config)
job.result() 

# Make an API request to upload the table
table = client.get_table(table_id)  
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 210216 rows and 5 columns to energy-ree-project.energy_generic_info.energy-demand-ree


## Bibliography
- Read multiple files into DataFrame: https://sparkbyexamples.com/pandas/pandas-read-multiple-csv-files/
- Website of the data: https://demanda.ree.es/visiona/peninsula/nacional/total/2020-08-01